In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import celerite as celery
import exoplanet as xo
import pymc3 as pm
import theano.tensor as tt
from exoplanet.gp import terms, GP

In [ ]:
xo.__version__

In [ ]:
np.random.seed(42)

#### generate RVs on a dense grid of times:

In [ ]:
log_S0 = -3.0
log_Q = 8.1
log_omega0 = -3.9

In [ ]:
kernel_onemode = celery.terms.SHOTerm(log_S0, log_Q, log_omega0)

In [ ]:
nights_per_year = 60
start_ts = np.arange(1,1+nights_per_year) * 3600. * 24. # nightly on consecutive nights
start_ts += np.random.randint(-60, 60, nights_per_year)*60. # random perturbation of up to an hour before/after
dt = 30. # grid spacing within a night (s)
N = 2. * 3600. / dt # 1 hours each night
t_all = np.ravel([np.arange(N) * dt + tt - 900. for tt in start_ts])

In [ ]:
gp = celery.GP(kernel_onemode)
gp.compute(t_all)
y_all = gp.sample()

In [ ]:
plt.plot(t_all, y_all)
plt.xlim([start_ts[0], start_ts[0]+3600.]);

#### bin down to simulate longer exposures:

In [ ]:
from utils import simulate_exposure

t = []
y = []

exp_time = 3. * 60.
n_exp = 10

for st in start_ts:
    inds = (t_all > st - 900.) & (t_all < st + exp_time+n_exp + 3600.)
    for n in range(n_exp):
        stt = st + n*exp_time # start time for this exposure, assumes no read time
        y.append(simulate_exposure(t_all[inds], y_all[inds], stt, exp_time))
        t.append(stt + exp_time/2.)

t = np.array(t)
y = np.array(y)

In [ ]:
# add photon noise:
noise_amp = 0.3
yerr = np.zeros_like(y) + noise_amp
y += np.random.normal(0., noise_amp, len(y))

In [ ]:
plt.plot(t_all, y_all)
plt.scatter(t,y)
plt.xlim([start_ts[0], start_ts[0]+1800.]);

#### model with an integrated kernel:

In [ ]:
with pm.Model() as model:
    kernel = terms.SHOTerm(log_S0=log_S0, log_w0=log_omega0, log_Q=log_Q)
    kernel_int = terms.IntegratedTerm(kernel, exp_time)
    gp = GP(kernel_int, t, yerr ** 2)
    gp.condition(y)
    mu, var = xo.eval_in_model(gp.predict(t_all, return_var=True))
    sd = np.sqrt(var)
    y_pred = xo.eval_in_model(gp.predict(t))

In [ ]:
plt.plot(t_all, y_all)
plt.scatter(t,y)
art = plt.fill_between(t_all, mu + sd, mu - sd, color="C1", alpha=0.3)
art.set_edgecolor("none")
plt.plot(t_all, mu, color="C1")
plt.xlim([start_ts[0], start_ts[0]+1800.]);